# This file is to generate the proximity matricies; Data Agnostic
- Helpful to just change a few lines of code where to import the data from. 

In [21]:
# Load up data

#& Imports
import sys
sys.path.insert(0, '/yunity/arusty/PF-GAP')
import os

#Import libraries, MDS from SKlearn, and all the custom files
import pandas as pd
from Pipeline.functions import *

#Functions
from QGAP.qgap import QGAP
from Redcomets.Redcomets import REDCOMETS
from RFGAP_Rocket.RFGAP_Rocket import RFGAP_Rocket
from RDST.rdst import RDST_GAP

#& Data Loading
time_series = pd.read_csv("/yunity/arusty/PF-GAP/data/ftse_100_close_prices.csv", index_col=0)
static_train =  None
#static_test = None
labels = np.array(pd.read_csv("/yunity/arusty/PF-GAP/data/ftse_100_sectors.csv")).flatten()
data_dir = "../../data/ftse_100/results/"


In [28]:
time_series.shape, labels.shape, static_train.shape if static_train is not None else "No static data"

((100, 1007), (100,), 'No static data')

# Generating the proximities

In [24]:
print("Beggining QGAP...")
qgap = QGAP(matrix_type="dense", interval_depth = 8, quantile_divisor = 8)
quant_prox = data_to_proximities(qgap, time_series, labels, static_train)
np.save(os.path.join(data_dir, "quant_prox.npy"), quant_prox)
print("---- QGAP Finished")
print("---- OOB Score: ", qgap._estimator.oob_score_)

Beggining QGAP...
---- QGAP Finished
---- OOB Score:  0.19


In [ ]:
print("Beggining Redcomets...")
if static_train is None:
    redcomets = REDCOMETS(variant = 3, perc_length = 0.7, n_trees = 100) 
else:
    redcomets = REDCOMETS(variant = 3, perc_length = 0.7, n_trees = 100, static_data=static_train)
redcomets_prox = data_to_proximities(redcomets, time_series, labels)
np.save(os.path.join(data_dir, "redcomets_prox.npy"), redcomets_prox)
print("---- Redcomets Finished")
print("---- OOB Score: ", redcomets.get_ensemble_oob_score())

Beggining Redcomets...
---- Redcomets Finished
---- OOB Score:  0.910604166893765


In [30]:
print("Beggining RFGAP-Rockets...")
rf_rocket = RFGAP_Rocket(prediction_type = "classification", rocket = "Multi",
                         n_kernels=256) # Rocket Kwargs
rocket_prox = data_to_proximities(rf_rocket, time_series, labels, static_train)
np.save(os.path.join(data_dir, "rocket_prox.npy"), rocket_prox)
print("---- RFGAP-Rockets Finished")
print("---- OOB Score: ", rf_rocket.rf_gap.oob_score_)

Beggining RFGAP-Rockets...
---- RFGAP-Rockets Finished
---- OOB Score:  0.16


In [31]:
print("Beggining RDST...")
rdst = RDST_GAP(save_transformed_data = True, max_shapelets = 10000, 
                shapelet_lengths = None, alpha_similarity = 0.3)
rdst_prox = data_to_proximities(rdst, time_series, labels, static_train)

np.save(os.path.join(data_dir, "rdst_prox.npy"), rdst_prox)
print("---- RDST Finished")
print("---- OOB Score: ", rdst._estimator.oob_score_)


Beggining RDST...
---- RDST Finished
---- OOB Score:  0.19


In [32]:
print("Beggining Fresh Prince...")

# This requires a channel dimension
time_series_reshaped = np.expand_dims(time_series.values, axis=1)

from FreshPrince.FreshPrince import FreshPRINCE_GAP
fresh_prince = FreshPRINCE_GAP(default_fc_parameters="minimal", n_estimators=200)
fresh_prince_prox = data_to_proximities(fresh_prince, time_series_reshaped, labels, static_train)

np.save(os.path.join(data_dir, "fresh_prince_prox.npy"), fresh_prince_prox)
print("---- Fresh Prince Finished")

try:
    print("---- OOB Score: ", fresh_prince._estimator.oob_score_)
except:
    print("---- OOB Score: Not available for Fresh Prince")

Beggining Fresh Prince...
---- Fresh Prince Finished
---- OOB Score: Not available for Fresh Prince


In [ ]:
print("Beggining PyF-GAP...")
sys.path.insert(0, '/yunity/arusty/PF-GAP/PFGAP')

#Check data shape to see if too large
if time_series.shape[1] > 500:
    print("---- PyF-GAP is not designed for large datasets, consider using a smaller dataset.")
    print("---- Truncating time series to 500 features.\n\n")
    time_series = time_series.iloc[:, -500:]
    
from PFGAP.PyPFGAP import PyPFGAP
pf = PyPFGAP()
pf_prox = data_to_proximities(pf, time_series, pd.Categorical(labels).codes.astype(str))

np.save(os.path.join(data_dir, "pf_prox.npy"), pf_prox)
print("\n\n---- PyF-GAP Finished")
print("---- OOB Score: Not available for PyF-GAP")

Beggining PyF-GAP...
---- PyF-GAP is not designed for large datasets, consider using a smaller dataset.
---- Truncating time series to 500 features.


reading file [train]:finished in 0:0:0.089
reading file [test]:finished in 0:0:0.025
Running on configurations...
Dataset: train, Training Data : 100x500 , Testing Data: 100x500, Train #Classes: 43, Test #Classes: 43
Repeats: 1 , Trees: 18 , Candidates per Split(r): 5
Output Dir: output, Export: 1, Verbosity: 1
Select DM per node: true , Shuffle Data: false, JVM WarmUp: false
----------------------------------------------------------------------------------------------------

-----------------Repetition No: 1 (train)   -----------------
Using: 4 MB, Free: 76 MB, Allocated Pool: 80 MB, Max Available: 1024 MB
core.ProximityForestResult@4f3f5b24
0.1.2.3.4.5.6.7.8.9.10.11.12.